# Running a CAT based on a synthetic correct/incorrect MIRT model

This example shows how to run a CAT based on a synthetic correct/incorrect
MIRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: DRuleItemCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
dims = 2
using ComputerAdaptiveTesting.DummyData: dummy_full, std_mv_normal, SimpleItemBankSpec, StdModel4PL
using ComputerAdaptiveTesting.MathTraits
using ComputerAdaptiveTesting.Responses: BooleanResponse

TODO: pass in dims

In [3]:
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel4PL(), VectorContinuousDomain(), BooleanResponse()),
    dims;
    num_questions=10,
    num_testees=2
)

(ComputerAdaptiveTesting.ItemBanks.SlipItemBank{ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}}([0.0, 0.1382343282990993, 0.0, 0.24494674208271242, 0.0, 0.13020393807461458, 0.14363746749942052, 0.02580369047538554, 0.15569974619691096, 0.14481295169542704], ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}([0.0, 0.0, 0.0, 0.0, 0.0, 0.22404360058473968, 0.08514652908113546, 0.0, 0.17792965885626957, 0.0], ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [-2.7652425945226358, -0.5631476621103746, 1.29189925613477, 0.27360227061343195, 0.4875893874936683, 1.2588017877283026, 1.2795740933509492, -0.9206574648320474, 0.9303992123396928, -2.2329610695734865], [0.9421565792307782 1.0806530110459058 … 1.2137067911403459 0.736775201926

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [4]:
max_questions = 9
integrator = CubaIntegrator([-6.0, -6.0], [6.0, 6.0], CubaVegas(); rtol=1e-2)
ability_estimator = MeanAbilityEstimator(PriorAbilityEstimator(std_mv_normal(dims)), integrator)
rules = CatRules(
    ability_estimator,
    DRuleItemCriterion(ability_estimator),
    FixedItemsTerminationCondition(max_questions)
)

CatRules
  next_item: ComputerAdaptiveTesting.NextItemRules.ItemStrategyNextItemRule{ComputerAdaptiveTesting.NextItemRules.ExhaustiveSearch1Ply, ComputerAdaptiveTesting.NextItemRules.DRuleItemCriterion{ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integrators.CubaIntegrator{ComputerAdaptiveTesting.Integrators.CubaVegas, Base.Pairs{Symbol, Float64, Tuple{Symbol}, NamedTuple{(:rtol,), Tuple{Float64}}}}}}}}
  termination_condition: ComputerAdaptiveTesting.TerminationConditions.FixedItemsTerminationCondition
  ability_estimator: ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integra

XXX: We shouldn't need to specify xs here since the distributions are not used -- rework

In [5]:
points = 3
xs = repeat(range(-2.5, 2.5, length=points)', dims, 1)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator, abilities)
for testee_idx in axes(responses, 2)
    @debug "Running for testee" testee_idx
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[:, testee_idx]
    abs_err = sum(abs.(θ .- true_θ))
    @info "convergence" true_θ θ abs_err
end

┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     -0.6721282008662874
│     -1.5196332773250285
│   θ =
│    2-element Vector{Float64}:
│     -0.4947656999711673
│     -0.5770541175102035
└   abs_err = 1.119941660709945
┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│      1.5586527910631833
│     -0.4253898552670672
│   θ =
│    2-element Vector{Float64}:
│     0.030882863128225992
│     0.640665360989618
└   abs_err = 2.5938251441916425


Make a plot showing how the estimated value converges during the CAT.

In [6]:
conv_lines_fig = ability_convergence_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*